In [1]:
%load_ext autoreload
%autoreload 2
from ASD_Circuits import *

In [2]:
g = LoadConnectome2()

In [ ]:
meta_jon = pd.read_csv("dat/Jon_data/exp_bias-match-specific.csv")
meta_jon.columns = ["STR", "EFFECT", "Rank", "NGene"]
meta_jon = meta_jon.set_index("STR")
meta_jon.head(2)

#### Directional Scoring

In [ ]:
Scores_nonweight = pd.read_csv("dat/Conn.Nonweight.csv", index_col=0)
Scores_weight = pd.read_csv("dat/Conn.Weight.csv", index_col=0)

In [ ]:
Scores_nonweight.head(2)

In [ ]:
def ScoreConn(STRs, Scores_nonweight):
    Total_Score = 0
    for i, STR1 in enumerate(STRs):
        for j, STR2 in enumerate(STRs):
            Total_Score += Scores_nonweight.loc[STR1, STR2] / np.sum(Scores_nonweight.loc[STR1, :])
    return Total_Score/len(STRs)
def ArgMaxConn(STRs, Scores_nonweight):
    to_remove = None
    max_score = 0
    for STR in STRs:
        xx = np.delete(STRs, np.where(STRs == STR))
        score = ScoreConn(xx, Scores_nonweight)
        #print(STR, score)
        if score > max_score:
            max_score = score
            to_remove = STR
    return to_remove, max_score

In [ ]:
InitSTRs = meta_jon.head(50).index.values
Scores = []
for i in range(len(InitSTRs), 1, -1):
    to_remove, score = ArgMaxConn(InitSTRs, Scores_nonweight)
    print(to_remove)
    Scores.append(score)
    InitSTRs = np.delete(InitSTRs, np.where(InitSTRs == to_remove))

In [ ]:
print(len(Scores))

In [ ]:
Scores

In [ ]:
plt.plot(list(range(len(InitSTRs), 2, -1)), Scores)

In [ ]:
class MostCohesiveCirtuis(Annealer):
    def __init__(self, Graph, InCirtuitNodes, OutCirtuitNodes):
        self.Graph = Graph
        #self.CandidateNodes = CandidateNodes
        #self.InCirtuitNodes = CandidateNodes
        #self.OutCirtuitNodes = []
        super(MostCohesiveCirtuis, self).__init__(state)  # important!
    def move(self):
        # Add node or Remove node
        initial_energy = self.energy()
        CandidateNodes
        FLAG_Drop = True
        if len(self.OutCirtuitNodes) == 0 or np.random.rand() > 0.7:
            FLAG_Drop = True
        else:
            FLAG_Drop = False
        if FLAG_Drop: # Drop a node
            node = random.choice(self.InCirtuitNodes)
            self.InCirtuitNodes.remove(node)
            self.OutCirtuitNodes.append(node)
        else: # Add a node
            node = random.choice(self.OutCirtuitNodes)
            self.OutCirtuitNodes.remove(node)
            self.InCirtuitNodes.append(node)
        return self.energy() - initial_energy
    def energy(self):
        top_nodes = self.Graph.vs.select(label_in=self.InCirtuitNodes)
        g2 = self.Graph.copy()
        g2 = g2.subgraph(top_nodes)
        cohesives = []
        for v in g2.vs:
            coh = InOutCohesiveSingleNode(self.Graph, g2, v["label"])
            cohesives.append(coh)
        cohesive = np.mean(cohesives)
        return 1 - cohesive

#### SA 

In [ ]:
#g = LoadConnectome2(ConnFil=
#                    "../dat/allen-mouse-conn/norm_density-max_ipsi_contra-pval_0.05-deg_min_1-by_weight.csv")
g = LoadConnectome2(ConnFil=
                    "../dat/allen-mouse-conn/norm_density-max_ipsi_contra-pval_0.05-deg_min_1-by_weight_pvalue.csv")
EdgeWeightsDict = EdgeDict(g, keyon="label")

In [ ]:
Candidate = meta_jon.head(50).index.values
ScoreSTRSet(g, Candidate, EdgeWeightsDict)

In [ ]:
def onexxx(meta_jon, N, g=g, EdgeWeightsDict=EdgeWeightsDict):
    CandidateNodes = meta_jon.head(N).index.values
    xxx = ScoreSTRSet(g, CandidateNodes, EdgeWeightsDict)
    YYY = []
    for i in range(100):
        CandidateNodes = np.random.choice(meta_jon.index.values, N, replace=False)
        yyy = ScoreSTRSet(g, CandidateNodes, EdgeWeightsDict)
        YYY.append(yyy)
    return xxx, np.mean(YYY)
XXX1, YYY1 = [], []
for i in range(213):
    xxx, yyy = onexxx(meta_jon, i)
    XXX1.append(xxx)
    YYY1.append(yyy)

In [ ]:
Candidate = meta_jon.head(50).index.values
ScoreSTRSet(g, Candidate, EdgeWeightsDict, Direction=True, Weighted=True)

In [ ]:
def onexxx(meta_jon, N, g=g, EdgeWeightsDict=EdgeWeightsDict):
    CandidateNodes = meta_jon.head(N).index.values
    xxx = ScoreSTRSet(g, CandidateNodes, EdgeWeightsDict, Direction=True, Weighted=True)
    YYY = []
    for i in range(100):
        CandidateNodes = np.random.choice(meta_jon.index.values, N, replace=False)
        yyy = ScoreSTRSet(g, CandidateNodes, EdgeWeightsDict, Direction=True, Weighted=True)
        YYY.append(yyy)
    return xxx, np.mean(YYY)
XXX2, YYY2 = [], []
for i in range(213):
    xxx, yyy = onexxx(meta_jon, i)
    XXX2.append(xxx)
    YYY2.append(yyy)

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), XXX1[1:213], label="ASD", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), YYY1[1:213], label="shuffle", color="green", linestyle="dotted")
ax.plot(list(range(1,213,1)), XXX2[1:213], label="ASD W&D", color="blue", linestyle="-")
ax.plot(list(range(1,213,1)), YYY2[1:213], label="shuffle W&D", color="blue", linestyle="dotted")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
plt.grid(True)
plt.xlabel("topN")
plt.ylabel("Cohe")

In [ ]:
def movingAVG(Input, smoothLen=3):
    res = []
    for i in range(len(Input)):
        if i - smoothLen < 0:
            new = np.mean(Input[0:i+smoothLen])
        elif i+ smoothLen > len(Input):
            new = np.mean(Input[i-smoothLen:])
        else:
            new = np.mean(Input[i-smoothLen:i+smoothLen])
        res.append(new)
    return res

In [ ]:
unweight = movingAVG(np.array(XXX1[1:213])/np.array(YYY1[1:213]))
weighted = movingAVG(np.array(XXX2[1:213])/np.array(YYY2[1:213]))

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), unweight, label="ASD", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), weighted, label="ASD W&D", color="blue", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
#plt.title("Ratio of ASD/Exp Cohesiveness")
plt.grid(True)
plt.xlabel("Top N as Candidate Structures")
plt.ylabel("ASD / Random Cohesiveness")

In [ ]:
fig, ax = plt.subplots(dpi=200)
ax.plot(list(range(1,213,1)), np.array(XXX1[1:213])-np.array(YYY1[1:213]), label="ASD", color="green", linestyle="-")
ax.plot(list(range(1,213,1)), np.array(XXX2[1:213])-np.array(YYY2[1:213]), label="ASD W&D", color="blue", linestyle="-")
ax.set_xlim(213, 1)  # decreasing time
ax.legend()
plt.title("Diff of ASD-Exp Cohesiveness")
plt.grid(True)
plt.xlabel("topN")
plt.ylabel("Cohe Diff")

In [ ]:
CandidateNodes = meta_jon.head(50).index.values
Init_States = np.ones(len(CandidateNodes))
for idx in range(len(Init_States)):
    if np.random.rand() > 0.5:
        Init_States[idx] = 0
ins = MostCohesiveCirtuis(Init_States, g, CandidateNodes, EdgeWeightsDict)

In [ ]:
%%time
InCirtuitNodes = meta_jon.head(50).index.values
top_nodes = g.vs.select(label_in=InCirtuitNodes)
g2 = g.copy().subgraph(top_nodes)
Node="Dentate_gyrus"
CohesivenessSingleNodeMaxInOut(g, g2, Node, EdgeWeightsDict)

In [ ]:
%%time
ins.copy_strategy = "deepcopy"
ins.Tmax=1
ins.Tmin=0.0001
Tmps, Energys, state, e = ins.anneal()
print("Done")

In [ ]:
plt.plot(Tmps, Energys)
plt.xlabel("Temp")
plt.ylabel("Energy")
plt.gca().invert_xaxis()
plt.xscale("log")
plt.show()

In [ ]:
print(1-e, state)

In [ ]:
InCirtuitNodes = ins.CandidateNodes[np.where(ins.state==1)[0]]
print(list(InCirtuitNodes))

In [ ]:
OutCirtuitNodes = ins.CandidateNodes[np.where(ins.state==0)[0]]
print(len(OutCirtuitNodes))
print(list(OutCirtuitNodes))

In [ ]:
top_nodes = g.vs.select(label_in=InCirtuitNodes)
g2 = g.copy().subgraph(top_nodes)

In [ ]:
############### developing plot circuits

In [ ]:
InCirtuitNodes = meta_jon.head(50).index.values
top_nodes = g.vs.select(label_in=InCirtuitNodes)
g2 = g.copy().subgraph(top_nodes)
Node="Prelimbic_area"
CohesivenessSingleNodeMaxInOut(g, g2, Node, EdgeWeightsDict)

In [ ]:
coord_dict = {}
for Node in InCirtuitNodes:
    Cohe, ITR = CohesivenessSingleNodeMaxInOut(g, g2, Node, EdgeWeightsDict)
    coord_dict[Node] = (Cohe, np.log10(min(10, ITR)))
    #print(Node, Cohe, ITR)
    #print(Node)

In [ ]:
STR2REG, REG2STR = LoadSTR2REG()

In [ ]:
Regions = list(REG2STR.keys())
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
r = random.random
color_dict = {}
i=0
for region in Regions:
    if i > len(colors)-1:
        i = 0
    color_dict[region] = colors[i]
    i += 1

In [ ]:
fig, ax = plt.subplots(dpi=200, figsize=(12,12))
for K,v in coord_dict.items():
    x,y = v
    ax.scatter(x,y, s=80, c=color_dict[STR2REG[K]], marker="o")
    ax.text(x,y,s=K)
factor = 0.98
for e in g2.es:
    #print(e.source, e.target)
    #print(g2.vs[e.source]["label"], g2.vs[e.target]["label"])
    point1 = coord_dict[g2.vs[e.source]["label"]]
    point2 = coord_dict[g2.vs[e.target]["label"]]
    plt.arrow(point1[0], point1[1], 
              dx=(point2[0]-point1[0]) * factor, dy=(point2[1]-point1[1]) * factor, 
              head_width = 0.01, width = 0.00001, ec="grey", alpha=0.6)
#plt.yscale("log")
plt.xlabel("Cohesiveness")
plt.ylabel("log(ITR)")
plt.show()

In [ ]:
############### End developing plot circuits

In [ ]:
CandidateNodes = meta_jon.head(100).index.values
Init_States = np.ones(len(CandidateNodes))
for idx in range(len(Init_States)):
    if np.random.rand() > 0.5:
        Init_States[idx] = 0
ins2 = MostCohesiveCirtuis(Init_States, g, CandidateNodes, EdgeWeightsDict)

In [ ]:
len(Init_States)

In [ ]:
%%time
ins2.copy_strategy = "deepcopy"
ins2.Tmax=1
ins2.Tmin=0.00001
Tmps, Energys, state, e = ins2.anneal()
print("Done")

In [ ]:
print(1-e, state)
OutCirtuitNodes = ins2.CandidateNodes[np.where(ins.state==0)[0]]
print(list(OutCirtuitNodes))
print(len(OutCirtuitNodes))

In [ ]:
print(len(OutCirtuitNodes))

In [ ]:
InCirtuitNodes = ins2.CandidateNodes[np.where(ins2.state==1)[0]]
print(len(InCirtuitNodes))
list(InCirtuitNodes)

In [ ]:
with open("test.circuits.txt", 'wt') as fout:
    fout.write("Cohesiveness:{}\n".format(1-e))
    fout.write("InCirtuitNodes:" + ",".join(InCirtuitNodes) + "\n")
    fout.write("OutCirtuitNodes:" + ",".join(OutCirtuitNodes) + "\n")

In [ ]:
with open("/Users/jiayao//Work/ASD_Circuits/src/dat/Jon_data/test.txt", 'rt') as fin:
    In_Circuits_Jon = []
    for l in fin:
        l=l.strip()
        In_Circuits_Jon.append(l)
jon = copy.deepcopy(ins)

In [ ]:
Out_Circuits_Jon = list()

In [ ]:
jon.state = np.zeros(50)
for Node in In_Circuits_Jon:
    jon.state[np.where(ins.CandidateNodes==Node)[0]] = 1
1 - jon.energy()

In [ ]:
1 - ins.energy()

In [ ]:
jon.energy() - ins.energy()

In [ ]:
#ins.state[np.where(ins.CandidateNodes=="Caudoputamen")[0]] = 1
#ins.state[np.where(ins.CandidateNodes=="Parafascicular_nucleus")[0]] = 1
#1 - ins.energy()

In [ ]:
#ins.state = np.ones(len(CandidateNodes))
#1 - ins.energy()

In [ ]:
def run_one_sim(meta_jon, g):
    CandidateNodes = np.random.choice(meta_jon.head(213).index.values, 100)
    Init_States = np.ones(len(CandidateNodes))
    for idx in range(len(Init_States)):
        if np.random.rand() > 0.5:
            Init_States[idx] = 0
    ins = MostCohesiveCirtuis(Init_States, g, CandidateNodes, None)
    ins.copy_strategy = "deepcopy"
    ins.Tmax=1
    ins.Tmin=0.00001
    Tmps, Energys, state, e = ins.anneal()
    return 1-e

res = []
for i in range(10):
    cohe = run_one_sim(meta_jon, g)
    print(cohe)
    res.append(cohe)

### Most Cohesive in connectome

In [ ]:
CandidateNodes = np.random.choice(meta_jon.head(213).index.values, 50)
Init_States = np.ones(len(CandidateNodes))
for idx in range(len(Init_States)):
    if np.random.rand() > 0.5:
        Init_States[idx] = 0
#print(Init_States)
#Init_States = jon.state
#del ins
ins = MostCohesiveCirtuis(Init_States, g, CandidateNodes)

In [ ]:
%%time
ins.copy_strategy = "deepcopy"
ins.Tmax=1
ins.Tmin=0.00001
Tmps, Energys, state, e = ins.anneal()
print("Done")

In [ ]:
plt.plot(Tmps, Energys)
plt.xlabel("Temp")
plt.ylabel("Energy")
plt.gca().invert_xaxis()
plt.xscale("log")
plt.show()

In [ ]:
print(1-e, state)

In [ ]:
def run_one_sim(meta_jon, g):
    CandidateNodes = np.random.choice(meta_jon.head(213).index.values, 50)
    Init_States = np.ones(len(CandidateNodes))
    for idx in range(len(Init_States)):
        if np.random.rand() > 0.5:
            Init_States[idx] = 0
    ins = MostCohesiveCirtuis(Init_States, g, CandidateNodes)
    ins.copy_strategy = "deepcopy"
    ins.Tmax=1
    ins.Tmin=0.00001
    Tmps, Energys, state, e = ins.anneal()
    return 1-e

res = []
for i in range(10):
    cohe = run_one_sim(meta_jon, g)
    res.append(cohe)

In [ ]:
print(np.mean(res))
print(res)